# Hotel Recommendation System(ML Assignment)


In [1]:
%pip install keras.layers.core

ERROR: Could not find a version that satisfies the requirement keras.layers.core (from versions: none)
ERROR: No matching distribution found for keras.layers.core
Note: you may need to restart the kernel to use updated packages.


# COD3

In [2]:
pip install pipreqs


  Using cached pipreqs-0.5.0-py3-none-any.whl (33 kB)
  Using cached yarg-0.1.9-py2.py3-none-any.whl (19 kB)
  Using cached ipython-8.12.3-py3-none-any.whl (798 kB)
  Using cached nbconvert-7.16.4-py3-none-any.whl (257 kB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 KB 1.1 MB/s eta 0:00:00a 0:00:01
  Using cached tinycss2-1.3.0-py3-none-any.whl (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 KB 2.8 MB/s eta 0:00:00a 0:00:01
  Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 KB 7.8 MB/s eta 0:00:00
  Using cached mistune-3.0.2-py3-none-any.whl (47 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/12

In [9]:
pip install numpy

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 26.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 KB 30.6 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install -r requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 44.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 KB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 32.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 17.3 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install -r /Users/liza/Downloads/hotel-hotel-latest/backend/requirements.txt

Processing /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_690hlorvpy/croot/aiohttp_1670009554132/work
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_690hlorvpy/croot/aiohttp_1670009554132/work'

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.3/273.3 KB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.9/390.9 KB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 35.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 KB 9.9 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.0/227.0 MB 25.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 48.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 28.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 45.8 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 35.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 KB 20.9 MB/

In [37]:
pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
    

In [39]:
pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 22.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 53.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 KB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 14.2 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install math

ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [42]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-macosx_10_9_universal2.whl size=1592425 sha256=49e5a58a4ca367cab263a5aed9fdcd910b4ed00b772320a1033ad9d845d06722
  Stored in directory: /Users/liza/Library/Caches/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [45]:
pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 2.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from keras.models import Sequential
# from keras.layers.core import Dense,Activation,Dropout
# from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from keras.callbacks import EarlyStopping
import math
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from scipy.sparse import csr_matrix

In [3]:
hotel_details=pd.read_excel('/Users/liza/Downloads/hotel-hotel-latest/backend/hotels.xlsx')


In [4]:
hotel_rooms=pd.read_excel('/Users/liza/Downloads/hotel-hotel-latest/backend/room.xlsx')

In [5]:
events_df = pd.read_excel('/Users/liza/Downloads/hotel-hotel-latest/backend/orders.xlsx')

# EDA

In [6]:
hotel=pd.merge(hotel_rooms,hotel_details,left_on='hotel_id',right_on='id',how='inner')

In [7]:

hotel = hotel.merge(events_df, on='hotel_id', how='left')

In [8]:
hotel = hotel.dropna()

In [9]:
del hotel['id_x']
del hotel['id_y']


In [126]:
hotel['description']=hotel['description'].str.replace(': ;',',')

In [10]:
hotel.hotel_id.unique

<bound method Series.unique of 0             31
1             31
2             31
3             31
4             31
            ... 
16792874    1534
16792875    1534
16792876    1534
16792877    1534
16792878    1534
Name: hotel_id, Length: 16792244, dtype: int64>

# Рекомендательная система, основанная только на городе и рейтингах отеля

In [6]:
def requirementbased(country, number, features):
    # Чтение данных из Excel файлов с информацией о гостиницах, номерах и заказах
    hotel_details = pd.read_excel('/Users/liza/Downloads/hotel-hotel-latest/backend/hotels.xlsx')
    hotel_rooms = pd.read_excel('/Users/liza/Downloads/hotel-hotel-latest/backend/room.xlsx')
    events_df = pd.read_excel('/Users/liza/Downloads/hotel-hotel-latest/backend/orders.xlsx')
    
    # Объединение таблиц по идентификатору гостиницы
    hotel = pd.merge(hotel_rooms, hotel_details, left_on='hotel_id', right_on='id', how='inner')
    hotel = hotel.merge(events_df, on='hotel_id', how='left')
    
    # Удаление строк с отсутствующими значениями
    hotel = hotel.dropna()
    
    # Удаление лишних столбцов
    del hotel['id_x']
    del hotel['id_y']
    
    # Преобразование текстовых данных в нижний регистр
    hotel['description'] = hotel['description'].str.replace(': ;', ',')
    hotel['country'] = hotel['country'].str.lower()
    hotel['description'] = hotel['description'].str.lower()
    
    # Преобразование входных особенностей в нижний регистр и токенизация
    features = features.lower()
    features_tokens = word_tokenize(features)
    
    # Удаление стоп-слов и лемматизация особенностей
    '''1. sw = stopwords.words('english'): Здесь загружаются стоп-слова на английском языке из библиотеки NLTK. Стоп-слова — это слова, которые обычно не несут смысловой нагрузки и могут быть исключены из анализа текста.

2. lemm = WordNetLemmatizer(): Создается объект WordNetLemmatizer, который используется для лемматизации слов. Лемматизация — это процесс приведения слова к его базовой форме (лемме).

3. f1_set = {w for w in features_tokens if not w in sw}: Здесь создается множество f1_set, содержащее слова из features_tokens, которые не являются стоп-словами. features_tokens предположительно содержит токены (слова) текста для анализа.

4. f_set = set(): Создается пустое множество f_set, которое будет содержать лемматизированные версии слов из f1_set.

5. for se in f1_set:: Начинается цикл по элементам множества f1_set.

6. f_set.add(lemm.lemmatize(se)): Для каждого элемента se из f1_set происходит лемматизация с помощью lemm.lemmatize(se), после чего лемматизированное слово добавляется в множество f_set.

Таким образом, в результате выполнения этого кода мы получаем множество f_set, содержащее лемматизированные версии слов из features_tokens, исключая стоп-слова, которые представлены в переменной sw. Лемматизация помогает сократить разнообразие форм слова до их базовых форм для более точного анализа текста.'''
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set = set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    
    # Фильтрация данных по стране и максимальному количеству номеров
    reqbased = hotel[hotel['country'] == country.lower()]
    reqbased = reqbased[reqbased['max_count'] == number]
    reqbased = reqbased.set_index(np.arange(reqbased.shape[0]))
    
    l1 = []; l2 = []; cos = []
    
    # Вычисление сходства особенностей для каждой гостиницы
    '''Данная часть кода представляет собой цикл, который проходит по каждой строке данных в DataFrame reqbased и вычисляет показатель сходства между особенностями гостиницы (описанными в столбце 'description') и особенностями, указанными пользователем в переменной features.

1. temp_tokens = word_tokenize(reqbased['description'][i]): Для каждой строки в reqbased извлекаются текстовые данные из столбца 'description' и токенизируются, то есть разбиваются на отдельные слова.

2. temp1_set = {w for w in temp_tokens if not w in sw}: Создается множество temp1_set, содержащее уникальные слова из описания гостиницы, исключая стоп-слова (слова, которые обычно не несут смысловой нагрузки).

3. temp_set = set(): Создается пустое множество temp_set, которое будет содержать лемматизированные слова из описания гостиницы.

4. for se in temp1_set: temp_set.add(lemm.lemmatize(se)): Каждое слово из temp1_set лемматизируется (приводится к начальной форме) с использованием WordNetLemmatizer и добавляется в temp_set.

5. rvector = temp_set.intersection(f_set): Создается множество rvector, содержащее пересечение слов из описания гостиницы (temp_set) и особенностей, указанных пользователем (f_set).

6. cos.append(len(rvector)): Длина множества rvector (то есть количество слов, которые пересекаются) добавляется в список cos, который будет использоваться для оценки сходства между описанием гостиницы и требованиями пользователя.

Этот процесс повторяется для каждой строки данных в reqbased, чтобы вычислить показатель сходства особенностей для каждой гостиницы.'''

    for i in range(reqbased.shape[0]):
        temp_tokens = word_tokenize(reqbased['description'][i])
        temp1_set = {w for w in temp_tokens if not w in sw}
        temp_set = set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        cos.append(len(rvector))
    
    # Добавление столбца с показателем сходства к данным и сортировка по этому показателю
    reqbased['similarity'] = cos
    reqbased = reqbased.sort_values(by='similarity', ascending=False)
    
    # Удаление дубликатов гостиниц и возврат топ-10 результатов
    reqbased.drop_duplicates(subset='hotel_id', keep='first', inplace=True)
    return reqbased[['name', 'country_rus', 'stars', 'price', 'hotel_img', 'room_img', 'description', 'category']].head(10)

In [7]:
import pickle
Pkl_Filename = "Pickle_RL_Model.pkl"
with open(Pkl_Filename, 'wb') as file:
    pickle.dump(requirementbased, file)

In [8]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/liza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/liza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/liza/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
requirementbased('Italy',1,'I need a extra toilet and room should be completely air conditioned.I should have a bathrobe.')

,name,country_rus,stars,price,hotel_img,room_img,description,category
0,Hotel Singapore,Италия,3,23602,https://www.tourdom.ru/upload/iblock/7fa/7fab8...,https://static.tildacdn.com/tild3835-3863-4339...,"air conditioning,alarm clock,bathrobes,carpeti...",Studio King Suite
1635,Schlossberg,Италия,2,10546,https://guide-tours.ru/wp-content/uploads/2022...,https://dolphinhotel.ru/_cache/udb1-rec13-fiel...,"air conditioning,daily housekeeping,desk,free ...",Standard Single Room
426,Hotel Holiday,Италия,3,19824,https://otdih.nakubani.ru/m/catalogBig/65d384f...,https://standarthotel.com/upload/iblock/1ea/1e...,"air conditioning,free wi-fi in all rooms!,hair...",Single Standard
1029,Hotel Vedig,Италия,4,21148,https://tophotels.ru/icache/hotel_photos/1/880...,https://uniqhotel.ru/wp-content/uploads/intery...,"air conditioning,carpeting,closet,desk,free wi...",Single Room


# Рекомендательная система по ЦА пользователя

In [28]:
hotel['interaction_value'] = hotel['status_name'].apply(lambda x: 0.6 if x == 'order' else (0.8 if x == 'pay' else 0.4))


In [29]:
item_index_map = {item: idx for idx, item in enumerate(hotel['hotel_id'].unique())}
visitor_index_map = {visitor: idx for idx, visitor in enumerate(hotel['user_id'].unique())}
cluster_index_map = {cluster: idx for idx, cluster in enumerate(hotel['cluster'].unique())}

row_indices = [visitor_index_map[visitor] for visitor in hotel['user_id']]
col_indices = [item_index_map[item] for item in hotel['hotel_id']]
interaction_values = hotel['interaction_value']

In [30]:
interaction_matrix = csr_matrix((interaction_values, (row_indices, col_indices)),
                                 shape=(len(visitor_index_map), len(item_index_map)))


In [31]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(0, 1))  
data = Dataset.load_from_df(hotel[['user_id', 'hotel_id', 'interaction_value']], reader)

model = SVD()

cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()
model.fit(trainset)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0997  0.0997  0.0997  0.0997  0.1000  0.0998  0.0001  
MAE (testset)     0.0994  0.0995  0.0995  0.0995  0.0996  0.0995  0.0001  
Fit time          113.57  113.84  116.88  123.75  115.51  116.71  3.72    
Test time         48.27   53.95   32.94   44.42   33.34   42.59   8.29    


In [56]:
def get_top_n_recommendations(model, user_id, num_items, n=10):
    recommendations = []
    for item_id in range(1, num_items + 1):
        prediction = model.predict(user_id, item_id)
        recommendations.append((item_id, prediction.est))

    # Sort recommendations by estimated rating
    recommendations.sort(key=lambda x: x[1], reverse=True)

    top_n_recommendations = recommendations[:n]
    return top_n_recommendations

In [33]:
from surprise.dump import dump

model_filename = 'trained_model.pkl'
dump(model_filename, algo=model)

In [59]:
from surprise.dump import load

loaded_model = load('trained_model.pkl')[1]

In [67]:
user_id = hotel['user_id'].sample().values[0]

num_items = hotel['hotel_id'].nunique()
top_recommendations = get_top_n_recommendations(loaded_model, user_id, num_items, n=10)
print("Лучшие рекомендации для пользователей", user_id)
for item_id, estimated_rating in top_recommendations:
    print("Item:", item_id, "Предполагаемый рейтинг по продукту:", estimated_rating)

Лучшие рекомендации для пользователей 936143.0
Item: 1 Предполагаемый рейтинг по продукту: 0.7066467026053401
Item: 2 Предполагаемый рейтинг по продукту: 0.7066467026053401
Item: 3 Предполагаемый рейтинг по продукту: 0.7066467026053401
Item: 4 Предполагаемый рейтинг по продукту: 0.7066467026053401
Item: 5 Предполагаемый рейтинг по продукту: 0.7066467026053401
Item: 6 Предполагаемый рейтинг по продукту: 0.7066467026053401
Item: 7 Предполагаемый рейтинг по продукту: 0.7066467026053401
Item: 8 Предполагаемый рейтинг по продукту: 0.7066467026053401
Item: 9 Предполагаемый рейтинг по продукту: 0.7066467026053401
Item: 10 Предполагаемый рейтинг по продукту: 0.7066467026053401


In [65]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Подготовка данных
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(hotel[['user_id', 'hotel_id', 'status']], reader)


# Обучение модели
trainset, testset = train_test_split(data, test_size=0.2)
model = SVD()
model.fit(trainset)

# Оценка модели
predictions = model.test(testset)
accuracy.rmse(predictions)

# Предсказание рекомендаций для конкретного пользователя
user_id = int(hotel['user_id'].sample().values[0])
top_n = 10
recommendations = {}
for hotel_id in hotel['hotel_id'].unique():
    pred = model.predict(user_id, hotel_id)
    recommendations[hotel_id] = pred.est

top_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:top_n]
print(top_recommendations)



RMSE: 0.7099
[(31, 1), (97, 1), (138, 1), (150, 1), (287, 1), (378, 1), (515, 1), (597, 1), (652, 1), (654, 1)]


# Рекомендательная система коллаборативный метод 2.0

In [26]:
import numpy as np
from scipy.sparse import csr_matrix

import numpy as np
from scipy.sparse import lil_matrix

import numpy as np
from scipy.sparse import csr_matrix

import numpy as np # linear algebra
import pandas as pd
import scipy.sparse as sp
from scipy.sparse import vstack
from scipy import sparse
from scipy.sparse.linalg import spsolve
from subprocess import check_output
from sklearn.model_selection import train_test_split

from lightfm import LightFM
import scipy.sparse as sp
from scipy.sparse import vstack

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


# Recommendation lightFM

In [11]:
user_activity_count = dict()
for row in hotel.itertuples():
    if row.user_id not in user_activity_count:
        user_activity_count[row.user_id] = {'order': 0, 'pay': 0}
    if row.status_name == 'order':
        user_activity_count[row.user_id]['order'] += 1 
    elif row.status_name == 'pay':
        user_activity_count[row.user_id]['pay'] += 1

d = pd.DataFrame(user_activity_count)
dataframe = d.transpose()
# Диапазон активности
dataframe['activity'] = dataframe['order'] + dataframe['pay'] 
# Удаление пользователей с одним просмотром
cleaned_data = dataframe[dataframe['activity'] != 1]
# all_users содержит идентификаторы пользователей с более чем 1 активностью в отеле (400000)
all_users = set(cleaned_data.index.values)
all_items = set(hotel['hotel_id'])
# todo: необходимо очистить элементы, которые были просмотрены только один раз

user_id_to_index_mapping = {}
hotel_id_to_index_mapping = {}
vid = 0
iid = 0
for row in hotel.itertuples():
    if row.user_id in all_users and row.user_id not in user_id_to_index_mapping:
        user_id_to_index_mapping[row.user_id] = vid
        vid = vid + 1

    if row.hotel_id in all_items and row.hotel_id not in hotel_id_to_index_mapping:
        hotel_id_to_index_mapping[row.hotel_id] = iid
        iid = iid + 1

In [15]:
filtered_items = hotel[hotel.hotel_id.isin(all_items)]

In [18]:
filtered_items = filtered_items.drop_duplicates(['hotel_id','cluster'])
filtered_items.sort_values(by='hotel_id', inplace=True)
item_to_property_matrix = filtered_items[['hotel_id','cluster']]

In [19]:
useful_cols = list()
cols = item_to_property_matrix.columns
for col in cols:
    value = len(item_to_property_matrix[col].value_counts())
    if value < 50:
        useful_cols.insert(0, col)

32
5


In [20]:
item_to_property_matrix = item_to_property_matrix[useful_cols]

In [21]:
item_to_property_matrix_one_hot_sparse = pd.get_dummies(item_to_property_matrix)

In [22]:
from scipy.sparse import csr_matrix
item_to_property_matrix_sparse = csr_matrix(item_to_property_matrix_one_hot_sparse.values)

In [25]:
import numpy as np

# Создаем нулевую матрицу взаимодействий с размерами (количество пользователей, количество отелей)
num_users = len(user_id_to_index_mapping)
num_hotels = len(hotel_id_to_index_mapping)
interactions_matrix = np.zeros((num_users, num_hotels))

# Заполняем матрицу взаимодействий на основе данных из DataFrame hotel
for index, row in hotel.iterrows():
    user_id = row['user_id']
    hotel_id = row['hotel_id']
    interaction_type = row['status']  

    user_index = user_id_to_index_mapping[user_id]
    hotel_index = hotel_id_to_index_mapping[hotel_id]

    interactions_matrix[user_index, hotel_index] = interaction_type

# Выводим полученную матрицу взаимодействий
print(interactions_matrix)

[[1. 0. 0. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 2.]
 [0. 0. 0. ... 0. 0. 2.]
 [0. 0. 0. ... 0. 0. 1.]]


In [28]:
import scipy.sparse as sp

interactions_matrix_sparse = sp.coo_matrix(interactions_matrix)
item_to_property_matrix_sparse = sp.coo_matrix(item_to_property_matrix_sparse)
no_comp, lr, ep = 30, 0.01, 10 

model = LightFM(no_components=no_comp, learning_rate=lr, loss='warp')
model.fit(
    interactions_matrix_sparse,
    item_features=item_to_property_matrix_sparse,
    epochs=ep,
    num_threads=4,
    verbose=True
)

Epoch: 100%|██████████| 10/10 [00:00<00:00, 1841.30it/s]


In [38]:
# Функция предсказания для пользователя
def predict_top_hotels_for_user(user_id, num_items_to_predict):
    scores = model.predict(user_id, np.arange(interactions_matrix_sparse.shape[1]), item_features=item_to_property_matrix_sparse)
    top_items = np.argsort(-scores)[:num_items_to_predict]
    return top_items

# Пример использования функции предсказания
user_id = 5
num_items_to_predict = 5
top_hotels = predict_top_hotels_for_user(user_id, num_items_to_predict)
print("Top recommended hotels for user:", top_hotels)


Top recommended hotels for user: [0 1 2 3 4]
